In [2]:
import numpy as np
from tabulate import tabulate
from prettytable import ALL, FRAME, PrettyTable

rules = {
    "S": [["NP", "VP", 0.8]],
    "NP": [["time", 0.002], ["flies", 0.002], ["arrow", 0.002], ["DET", "NOMINAL", 0.3], ["NOMINAL", "NOMINAL", 0.2]],
    "NOMINAL": [["time", 0.002], ["flies", 0.002], ["arrow", 0.002], ["NOMINAL", "NOUN", 0.1], ["NOMINAL", "PP", 0.2]],
    "VP": [["time", 0.004], ["flies", 0.008], ["like", 0.008], ["VERB", "NP", 0.3], ["VERB", "PP", 0.2]],
    "PP": [["PREPOSITION", "NP", 0.1]],
    "VERB": [["time", 0.01], ["flies", 0.02], ["like", 0.02]],
    "NOUN": [["time", 0.01], ["flies", 0.01], ["arrow", 0.01]],
    "DET": [["an", 0.05]],
    "PREPOSITION": [["like", 0.05]]
    }

sentence = ["time", "flies", "like", "an", "arrow"]

In [35]:
# Función del algoritmo de CKY
def cky(sentence, rules):
  n = len(sentence)
  # Se crea matriz vacía
  table = [[set([]) for j in range(n)] for i in range(n)]

  for k in range(n):
    i = 0
    for j in range(k, n):
      # Llenar primera diagonal de lo que significa cada palabra por si sola
      if k == 0:
        for key, key_rules in rules.items():
          for rule in key_rules:
            if len(rule) == 2 and rule[0] == sentence[i]:
              build = key + ":[" + sentence[i] + "]"
              table[i][j].add((key, rule[-1], build))
      # Llenar resto de la matriz en la que se unen palabras
      else:
          for l in range(i, j):
            for label, prob, build in table[i][l]:

              for key, key_rules in rules.items():
                for rule in key_rules:
                  if len(rule) == 3 and rule[0] == label:
                    
                    for label2, prob2, build2 in table[l + 1][j]:
                      if rule[1] == label2:
                        new_build = key + ":[" + build + "+ " + build2 + "]"
                        table[i][j].add((key, rule[-1] * prob * prob2, new_build))
            
      i += 1


  # Imprimir estructura del árbol
  def print_tree():
    # Imprimir estructura de frase más probable
    results = table[0][n-1]
    max_S = 0.0
    final_build = ""
    for key, prob, build in results:
      if key == "S" and prob > max_S:
        final_build = build
        max_S = prob

    if max_S == 0.0:
      print("No se encontró una frase dada la oración")
    else:
      print("Estructura de frase más probable encontrada por el agente:")
      print("  ", final_build)
      print("  ", "Probabilidad:", max_S)

    spaces = 0
    st = ''
    for i in range(len(final_build)):
      if i == 0:
        print(final_build[i])
      else:
        if final_build[i] == '[':
          spaces += 2

        if final_build[i] == ']' or final_build[i] == ':':
          word = '|'
          for j in range(spaces):
            word += '-'
          word += st
          if st != '':
            print(word)
          st = ''

        if final_build[i] == ']':
          spaces -= 2

        if final_build[i] != ':' and final_build[i] != '+'and final_build[i] != '[' and final_build[i] != ']'  and final_build[i] != ' ':
          st += final_build[i]
    
  # Imprimir matriz
  def print_matrix(sentence):
    labels = [""]
    labels.extend(sentence)
    tab = PrettyTable(labels)
    for i in range(len(table)):
      labels = [sentence[i]]
      labels.extend(table[i])
      tab.add_row(labels)
    tab.int_format = '8'
    tab.padding_width = 2
    tab.hrules = ALL
    print(tab)

  print_matrix(sentence)
  print('\n\n')
  print_tree()

cky(sentence, rules)

+---------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------+-------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------